### Question: Predicting # of Flight delays given certain weather and past flight data

In [1]:
import pandas as pd
import numpy as np


In [2]:
# Read in csv files from Kaggle
pressure_df = pd.read_csv('weather_data/pressure.csv')
humidity_df = pd.read_csv('weather_data/humidity.csv')
temperature_df = pd.read_csv('weather_data/temperature.csv')
wind_direction_df = pd.read_csv('weather_data/wind_direction.csv')
wind_speed_df = pd.read_csv('weather_data/wind_speed.csv')
weather_description_df = pd.read_csv("weather_data/weather_description.csv")


In [3]:
# Top 5 cities with most values in flight delay dataset
cities = ["Atlanta",'Chicago','Dallas','Charlotte','Denver']

In [4]:
def clean_df(dataframe,name):
    dataframe = dataframe[["datetime"] + cities] # call subset of df
    dataframe_2015_index = list(dataframe.index[dataframe['datetime'] == "2015-01-01 00:00:00"])
    # going from 2015 - 2017
    dataframe = dataframe.iloc[dataframe_2015_index[0]:,:]
    # add name to columns specifying originating df. Need this specified when joining table
    new_names = dict(zip(list(cities),list(np.core.defchararray.add(cities,name))))
    new_names['datetime'] = 'datetime'
    dataframe = dataframe.rename(columns = new_names)
    return dataframe
        

In [5]:
# New cleaned and correctly labeled df
pressure_df = clean_df(pressure_df, '_pressure')
humidity_df = clean_df(humidity_df, '_humidity')
temperature_df = clean_df(temperature_df, '_temperature')
wind_direction_df = clean_df(wind_direction_df, '_wind_direction')
wind_speed_df = clean_df(wind_speed_df, '_wind_speed')
weather_description_df = clean_df(weather_description_df, '_description')

In [6]:
# Merge the individual datasets on datetime
A = pd.merge(pressure_df.merge(humidity_df, how = 'outer'),temperature_df, how = 'outer')
B = pd.merge(wind_direction_df.merge(wind_speed_df, how = 'outer'), weather_description_df, how = 'outer')
weather_df = A.merge(B, how = 'outer')
    

In [7]:
weather_df.head()
# weather.columns

,datetime,Atlanta_pressure,Chicago_pressure,Dallas_pressure,Charlotte_pressure,Denver_pressure,Atlanta_humidity,Chicago_humidity,Dallas_humidity,Charlotte_humidity,...,Atlanta_wind_speed,Chicago_wind_speed,Dallas_wind_speed,Charlotte_wind_speed,Denver_wind_speed,Atlanta_description,Chicago_description,Dallas_description,Charlotte_description,Denver_description
0,2015-01-01 00:00:00,1042.0,1039.0,1047.0,1042.0,1046.0,79.0,100.0,76.0,81.0,...,1.0,9.0,4.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear
1,2015-01-01 01:00:00,1043.0,1037.0,1047.0,1042.0,1046.0,78.0,100.0,74.0,92.0,...,2.0,10.0,4.0,1.0,1.0,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear
2,2015-01-01 02:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,80.0,100.0,76.0,89.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear
3,2015-01-01 03:00:00,1043.0,1037.0,1046.0,1042.0,1046.0,78.0,100.0,75.0,91.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear
4,2015-01-01 04:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,81.0,100.0,78.0,86.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear


In [8]:
# Check for column names that have null values
null_list = list(weather_df.columns[weather_df.isna().any()])
for column in null_list:
    # check missing values ratio to dataset size
    print(column, (weather_df[column].isna().sum()/weather_df.shape[0])*100)
    
# Output:
# Chicago_pressure 0.003915886752555116
# Atlanta_humidity 0.07440184829854721
# Chicago_humidity 0.003915886752555116
# Dallas_humidity 0.12922426283431884
# Charlotte_humidity 0.775345577005913
# Denver_humidity 0.01174766025766535
# Dallas_wind_speed 0.003915886752555116
# Charlotte_wind_speed 0.003915886752555116

# Since they're so minute, will just drop rows that have null values

Chicago_pressure 0.003915886752555116
Atlanta_humidity 0.07440184829854721
Chicago_humidity 0.003915886752555116
Dallas_humidity 0.12922426283431884
Charlotte_humidity 0.775345577005913
Denver_humidity 0.01174766025766535
Dallas_wind_speed 0.003915886752555116
Charlotte_wind_speed 0.003915886752555116


In [9]:
weather_df.shape # 25537 rows
weather_df = weather_df.dropna()
weather_df.shape # 240 rows dropped

(25297, 31)

In [10]:
weather_df.head()

,datetime,Atlanta_pressure,Chicago_pressure,Dallas_pressure,Charlotte_pressure,Denver_pressure,Atlanta_humidity,Chicago_humidity,Dallas_humidity,Charlotte_humidity,...,Atlanta_wind_speed,Chicago_wind_speed,Dallas_wind_speed,Charlotte_wind_speed,Denver_wind_speed,Atlanta_description,Chicago_description,Dallas_description,Charlotte_description,Denver_description
0,2015-01-01 00:00:00,1042.0,1039.0,1047.0,1042.0,1046.0,79.0,100.0,76.0,81.0,...,1.0,9.0,4.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear
1,2015-01-01 01:00:00,1043.0,1037.0,1047.0,1042.0,1046.0,78.0,100.0,74.0,92.0,...,2.0,10.0,4.0,1.0,1.0,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear
2,2015-01-01 02:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,80.0,100.0,76.0,89.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear
3,2015-01-01 03:00:00,1043.0,1037.0,1046.0,1042.0,1046.0,78.0,100.0,75.0,91.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear
4,2015-01-01 04:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,81.0,100.0,78.0,86.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear


In [11]:
# Split datetime column into Years, Months, Days and time column
Year = []
Month = []
Day = []
Time = []
for datetime in weather_df['datetime']:
    date, time = datetime.split(' ')
    year, month, day = date.split('-')
    Year.append(year)
    Month.append(month)
    Day.append(day)
    Time.append(time)

# Convert arrays to pandas Series and make it int type, so when merging, the object-type will be same
weather_df['Year'] = pd.Series(Year, dtype='int32')
weather_df["Month"] = pd.Series(Month, dtype='int32')
weather_df['Day'] = pd.Series(Day, dtype='int32')
weather_df['Time'] = Time

In [12]:
# Rearranging them so date/time is in beginning
weather_df = weather_df[['Year', 'Month', 'Day','Time','Atlanta_pressure', 'Chicago_pressure', 'Dallas_pressure',\
       'Charlotte_pressure', 'Denver_pressure', 'Atlanta_humidity',\
       'Chicago_humidity', 'Dallas_humidity', 'Charlotte_humidity',\
       'Denver_humidity', 'Atlanta_temperature', 'Chicago_temperature',\
       'Dallas_temperature', 'Charlotte_temperature', 'Denver_temperature',\
       'Atlanta_wind_direction', 'Chicago_wind_direction',\
       'Dallas_wind_direction', 'Charlotte_wind_direction',\
       'Denver_wind_direction', 'Atlanta_wind_speed', 'Chicago_wind_speed',\
       'Dallas_wind_speed', 'Charlotte_wind_speed', 'Denver_wind_speed',\
       'Atlanta_description', 'Chicago_description', 'Dallas_description',\
       'Charlotte_description', 'Denver_description']]
weather_df.head()

,Year,Month,Day,Time,Atlanta_pressure,Chicago_pressure,Dallas_pressure,Charlotte_pressure,Denver_pressure,Atlanta_humidity,...,Atlanta_wind_speed,Chicago_wind_speed,Dallas_wind_speed,Charlotte_wind_speed,Denver_wind_speed,Atlanta_description,Chicago_description,Dallas_description,Charlotte_description,Denver_description
0,2015.0,1.0,1.0,00:00:00,1042.0,1039.0,1047.0,1042.0,1046.0,79.0,...,1.0,9.0,4.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear
1,2015.0,1.0,1.0,01:00:00,1043.0,1037.0,1047.0,1042.0,1046.0,78.0,...,2.0,10.0,4.0,1.0,1.0,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear
2,2015.0,1.0,1.0,02:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,80.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear
3,2015.0,1.0,1.0,03:00:00,1043.0,1037.0,1046.0,1042.0,1046.0,78.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear
4,2015.0,1.0,1.0,04:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,81.0,...,1.0,10.0,3.0,1.0,1.0,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear


In [48]:
weather_df['hour']= pd.to_datetime(weather_df['Time']).dt.hour
weather_df["DayofMonth"]=weather_df["Day"]
weather_df.columns

Index(['Year', 'Month', 'Day', 'Time', 'Atlanta_pressure', 'Chicago_pressure',
       'Dallas_pressure', 'Charlotte_pressure', 'Denver_pressure',
       'Atlanta_humidity', 'Chicago_humidity', 'Dallas_humidity',
       'Charlotte_humidity', 'Denver_humidity', 'Atlanta_temperature',
       'Chicago_temperature', 'Dallas_temperature', 'Charlotte_temperature',
       'Denver_temperature', 'Atlanta_wind_direction',
       'Chicago_wind_direction', 'Dallas_wind_direction',
       'Charlotte_wind_direction', 'Denver_wind_direction',
       'Atlanta_wind_speed', 'Chicago_wind_speed', 'Dallas_wind_speed',
       'Charlotte_wind_speed', 'Denver_wind_speed', 'Atlanta_description',
       'Chicago_description', 'Dallas_description', 'Charlotte_description',
       'Denver_description', 'hour', 'DayofMonth'],
      dtype='object')

In [50]:
#break up to have columsn year mondy time and pressure, humidyt, wind speed and Charlotte_description for EACH CITY TO concate to datalist 
#Add origin as the symbol of the airport 
#combine 
Atlanta1= weather_df[['Year', 'Month', 'DayofMonth', 'Time',"hour","Atlanta_pressure","Atlanta_humidity","Atlanta_wind_direction","Atlanta_description"]]
Chicago1=weather_df[['Year', 'Month', 'DayofMonth', 'Time',"hour","Chicago_pressure","Chicago_humidity","Chicago_wind_direction","Chicago_description"]]
Denver1=weather_df[['Year', 'Month', 'DayofMonth', 'Time',"hour","Denver_pressure","Denver_humidity","Denver_wind_direction","Denver_description"]]
Dallas1=weather_df[['Year', 'Month', 'DayofMonth', 'Time',"hour","Dallas_pressure","Dallas_humidity","Dallas_wind_direction","Dallas_description"]]
Charlotte1=weather_df[['Year', 'Month', 'DayofMonth', 'Time',"hour","Charlotte_pressure","Charlotte_humidity","Charlotte_wind_direction","Charlotte_description"]]


In [51]:
# Atlanta1["Origin"]=("ATL")
Atlanta1 = Atlanta1.assign(Origin="ATL")
Chicago1= Chicago1.assign(Origin="ORD")
Denver1= Denver1.assign(Origin="DEN")
Dallas1= Dallas1.assign(Origin="DFW")
Charlotte1= Charlotte1.assign(Origin="CTL")

In [54]:
Atlanta1
Chicago1
# (Denver1[Denver1["Year"]].isna())
print(len(Denver1["Year"])) #25297
Denver1["Year"].isnull().sum() #2240 
# Dallas1.tail()
# Charlotte1


25297


240

In [55]:
Atlanta1

,Year,Month,DayofMonth,Time,hour,Atlanta_pressure,Atlanta_humidity,Atlanta_wind_direction,Atlanta_description,Origin
0,2015.0,1.0,1.0,00:00:00,0,1042.0,79.0,344.0,sky is clear,ATL
1,2015.0,1.0,1.0,01:00:00,1,1043.0,78.0,342.0,sky is clear,ATL
2,2015.0,1.0,1.0,02:00:00,2,1043.0,80.0,342.0,sky is clear,ATL
3,2015.0,1.0,1.0,03:00:00,3,1043.0,78.0,324.0,sky is clear,ATL
4,2015.0,1.0,1.0,04:00:00,4,1043.0,81.0,330.0,sky is clear,ATL
5,2015.0,1.0,1.0,05:00:00,5,1043.0,80.0,317.0,few clouds,ATL
6,2015.0,1.0,1.0,06:00:00,6,1043.0,80.0,329.0,sky is clear,ATL
7,2015.0,1.0,1.0,07:00:00,7,1042.0,79.0,308.0,few clouds,ATL
8,2015.0,1.0,1.0,08:00:00,8,1042.0,79.0,312.0,sky is clear,ATL
9,2015.0,1.0,1.0,09:00:00,9,1042.0,78.0,311.0,sky is clear,ATL
